In [ ]:
import numpy as np
import scipp as sc
import scippneutron as scn
import urllib.request

url = 'https://github.com/ess-dmsc-dram/loki_tube_scripts/raw/master/test/test_data/LARMOR00049338.nxs'
filename, _ = urllib.request.urlretrieve(url)  # noqa: S310
data = scn.load_with_mantid(filename=filename)
edges = sc.array(dims=['tof'], unit='us', values=np.linspace(5.0, 100000.0, num=201))
data = sc.rebin(data, 'tof', edges)
for i in [1, 2, 3, 4, 5]:
    mon = data.attrs[f'monitor{i}']
    mon.value = sc.rebin(mon.value, 'tof', edges)
data.to_hdf5(filename='loki-at-larmor.hdf5')

In [ ]:
import urllib

url = 'http://198.74.56.37/ftp/external-data/MD5/d5ae38871d0a09a28ae01f85d969de1e'
filename, _ = urllib.request.urlretrieve(url, filename='PG3_4844_event.nxs')  # noqa: S310

In [ ]:
import scipp as sc
import scippneutron as scn

da = scn.load_with_mantid(filename='PG3_4844_event.nxs', load_pulse_times=True)

# Fake d-spacing shift
da = scn.convert(da, 'tof', 'dspacing', scatter=True)
proton_charge = da.attrs['proton_charge'].value
tmin = proton_charge.coords['time'].min()
tmax = proton_charge.coords['time'].max()
delta = sc.to_unit(tmax - tmin, 'us')
delta.unit = ''
scale = sc.to_unit(da.bins.coords['pulse_time'] - tmin, 'us') * sc.scalar(
    1, unit='Angstrom/us'
)
da.bins.coords['dspacing'] += 0.02 * scale / delta
da = scn.convert(da, 'dspacing', 'tof', scatter=True)
da.coords['tof'] = da.coords['tof']['spectrum', 0]

# Fake prompt pulse
prompt_start = 4000.0 * sc.Unit('us')
prompt_stop = 5000.0 * sc.Unit('us')
tof = da.bins.coords['tof']
da.bins.data *= sc.where(
    (prompt_start <= tof) & (tof < prompt_stop),
    1.0 + 3.0 * sc.exp(-(tof - prompt_start) / sc.scalar(200.0, unit='us')),
    sc.scalar(1.0),
)

# Reduce data size to 1/3
da['spectrum', 14100:].to_hdf5(filename='powder-event.h5')